# Notebook 20: Zero-Copy Memory## Phase 4: Advanced Memory & Synchronization**Learning Objectives:**- Understand zero-copy- Learn mapped memory- Master direct access- Apply concepts in practical scenarios- Measure and analyze performance

## Concept: Zero-Copy Memory**Topics Covered:**- zero-copy- mapped memory- direct access**Key Concepts:**This notebook covers zero-copy in the context of Phase 4: Advanced Memory & Synchronization.

## Example 1: Basic Zero-Copy Memory

In [ ]:
%%cu// Uses cudaHostAlloc mapped memory// Full implementation pending#include <stdio.h>int main() {    printf("=== Zero Copy ===\n");    printf("Topic: Uses cudaHostAlloc mapped memory\n");    printf("Implementation: See local .cu files for complete code\n");    return 0;}

## Practical ExerciseComplete the following exercises to practice the concepts learned.

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

__global__ void kernel(float *data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

int main() {
    printf("=== Zero Copy ===\n\n");

    int n = 1000000;
    size_t size = n * sizeof(float);

    float *h_data = (float*)malloc(size);
    for (int i = 0; i < n; i++) h_data[i] = i;

    float *d_data;
    CUDA_CHECK(cudaMalloc(&d_data, size));
    CUDA_CHECK(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int threads = 256;
    int blocks = (n + threads - 1) / threads;

    cudaEventRecord(start);
    kernel<<<blocks, threads>>>(d_data, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    CUDA_CHECK(cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost));

    printf("Processed %d elements in %.2f ms\n", n, ms);
    printf("Bandwidth: %.2f GB/s\n", (size * 2 / 1e9) / (ms / 1000.0));

    free(h_data);
    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

## Key Takeaways

1. Zero-copy memory maps host memory to device
2. Useful for PCIe-connected GPUs
3. No explicit cudaMemcpy needed
4. Trade-off: convenience vs bandwidth

## Next StepsContinue to: **21_next_topic.ipynb**

## Notes*Use this space to write your own notes and observations:*------